# **OntoGPT**
OntoGPT is a Python package for extracting structured information from text with large language models (LLMs), instruction prompts, and ontology-based grounding. It works well with OpenAI's GPT models as well as a selection of other LLMs. OntoGPT's output can be used for general-purpose natural language tasks (e.g., named entity recognition and relation extraction), summarization, knowledge base and knowledge graph construction, and more.

**Methods**
The primary extraction method currently implemented in OntoGPT is SPIRES:

SPIRES: Structured Prompt Interrogation and Recursive Extraction of Semantics
A Zero-shot learning (ZSL) approach to extracting nested semantic structures from text
This approach takes two inputs:1) LinkML schema  , 2) free text, and outputs knowledge in a structure conformant with the supplied schema in JSON, YAML, RDF or OWL formats

**Uses OpenAI GPT models through their API, or one of a variety of LLMs on your local machine**

You can run ollama server in the log in node as follwoing

module load ollama  
ollama serve  

* bind the local bin, where we installed OntoGPT to $PATH 

In [ ]:
%env PATH=../.local/bin:$PATH

* test OntoGPT

In [ ]:
!ontogpt extract --help

* test OntoGPT with Ollama.gemma  
Gemma is a family of lightweight, state-of-the-art open models built by Google DeepMind. Updated to version 1.1

In [ ]:
!ontogpt complete -m ollama/gemma:7b  "Why did the squid cross the coral reef?"

* We can extract knowledge from the  text  into the GO pathway datamodel by running the following command:

In [ ]:
!ontogpt extract -m ollama/gemma:7b -t gocam.GoCamAnnotations -i ../gocam-betacat.txt

Note: The value accepted by the -t / --template argument is the base name of one of the LinkML schema / data model available to OntoGPT.  

Use the command ontogpt list-templates to see all templates. Use the name in the first column with the --template option.  
The output returned from the above command can be optionally redirected into an output file using the -o / --output.  

In [ ]:
!ontogpt extract -m ollama/gemma:7b -t drug -i ../gocam-betacat.txt